In [1]:
from marple.datatypes import Domain
from marple.schema import Datatype
from marple.csv import CsvFile
from marple.utils import isNaN
import csvkit as csv
import os
from copy import deepcopy
import pprint
pp = pprint.PrettyPrinter(indent=2)


In [2]:
# Get from settings, should point to root folder
DATATYPES_DIR = ".."
# Get from request
LANG = "en"
# Get from request
DOMAIN = "http://marple-datatypes.herokuapp.com"

# Points to root folder
RELATIONS_CSV_PATH = "../relations.csv"

# Use this object for getting labels etc
# Get item by id: ALL_DOMAINS.row("Stockholms kommun")
# Get label for item: ALL_DOMAINS.label("Stockholms kommun", lang="en")
# Get parent for item: ALL_DOMAINS.parent("Stockholms kommun")
# Get children for item: ALL_DOMAINS.children("Stockholms kommun")

ALL_DOMAINS = Domain("*/*", datatypes_dir=DATATYPES_DIR)



In [3]:
def jsonify_item(item_id, lang, domain):
    """ 
    """
    return {
        "id": item_id,
        "label": ALL_DOMAINS.label(item_id, lang=lang),
        "path": u"{}/item/{}".format(domain, item_id),
    }

`/GET /datatype`

In [4]:
data = []

file_path = os.path.join(DATATYPES_DIR, "datatypes.csv")
csv_file = CsvFile(file_path)
# TODO: Add path
pp.pprint(csv_file.to_dictlist()[:5])

[ { u'allowed_values': u'misc/age_groups',
    u'description': None,
    'id': u'age_group',
    u'value_type': u'str'},
  { u'allowed_values': u'misc/crime_codes',
    u'description': u'A crime code (brottskod) used by the Swedish police',
    'id': u'crime_code',
    u'value_type': u'str'},
  { u'allowed_values': u'misc/crime_types',
    u'description': u'A crime type (brottstyp) used by the Swedish police. Higher level than brottskod.',
    'id': u'crime_type',
    u'value_type': u'str'},
  { u'allowed_values': u'misc/foreignborn',
    u'description': u'Is the subject born outside Sweden?',
    'id': u'foreignborn',
    u'value_type': u'str'},
  { u'allowed_values': u'misc/genders',
    u'description': u'A subjects gender',
    'id': u'gender',
    u'value_type': u'str'}]


`/GET /datatype/{datatype_id}`

In [5]:
datatype_id = "gender"
datatype = Datatype(datatype_id, datatypes_dir=DATATYPES_DIR)
allowed_values = []

data = {
    "id": datatype_id,
    "allowed_values": []
}

# Get lang and domain from request!
for allowed_value_id in datatype.allowed_values:
    data["allowed_values"].append(jsonify_item(allowed_value_id, LANG, DOMAIN))
    
pp.pprint(data)

{ 'allowed_values': [ { 'id': u'male',
                        'label': u'Men',
                        'path': u'http://marple-datatypes.herokuapp.com/item/male'},
                      { 'id': u'female',
                        'label': u'Women',
                        'path': u'http://marple-datatypes.herokuapp.com/item/female'},
                      { 'id': u'other',
                        'label': u'Other',
                        'path': u'http://marple-datatypes.herokuapp.com/item/other'},
                      { 'id': u'unknown',
                        'label': u'Unknown',
                        'path': u'http://marple-datatypes.herokuapp.com/item/unknown'},
                      { 'id': u'all genders',
                        'label': u'Both men and women',
                        'path': u'http://marple-datatypes.herokuapp.com/item/all genders'}],
  'id': 'gender'}


`/GET /item/{item_id}` 

In [6]:
def remove_nan(obj): 
    """Remove all NaN values
    """
    for key, value in obj.items():
        if isNaN(value):
            obj.pop(key, None)
    return obj

item_id = "Stockholms kommun"
# TODO: Use "**/*" to fetch all files in all subfolders once glob2 in place
data = ALL_DOMAINS.row(item_id)
# This is something of a hack. The Domain class will include a lot of empty columns/propeties defined in other files
# Here we clean up those
data = remove_nan(data)

# Translate label to selected language
data["label"] = ALL_DOMAINS.label(item_id,lang=LANG)

# Populate relational properties (parent, neighbours etc)
# These relations are defined in relations.csv in the root folder
relations_csv = CsvFile(RELATIONS_CSV_PATH)
# Ie {u'neighbours': u'one_to_many', u'parent': u'one_to_one'}
relational_columns = dict(relations_csv.data.to_records())

for column, relation_type in relational_columns.iteritems():
    if column not in data:
        continue
    if relation_type == "one_to_one":
        related_item_id = data[column] # ie the parent id
        data[column] = jsonify_item(related_item_id, LANG, DOMAIN)
        
    elif relation_type == "one_to_many":
        related_item_ids = data[column].split(",") #ie neighbours
        data[column] = []
        for related_item_id in related_item_ids:
            data[column].append(jsonify_item(related_item_id, LANG, DOMAIN))

data["children"] = []             
for child_id in all_domains.children(item_id):
    data["children"].append(jsonify_item(child_id, LANG, DOMAIN))

pp.pprint(data)

NameError: name 'all_domains' is not defined

In [ ]:
dict(relations_csv.data.to_records())